Deck class is to handle which cards are in the deck. Preventing duplicated cards to appear.

In [1]:
# Deck class
import sys
import random

sys.setrecursionlimit(3000)

class Deck():
    deck=[]
    clubs=list(zip(['clubs']*13,range(1,14)))
    diamonds=list(zip(['diamonds']*13,range(1,14)))
    hearts=list(zip(['hearts']*13,range(1,14)))
    spades=list(zip(['spades']*13,range(1,14)))
    
    def __init__(self):
        self.shuffle()
    
    def shuffle(self):
        self.deck= self.clubs + self.diamonds + self.hearts + self.spades
        random.shuffle(self.deck)
    
    def draw(self):
        #self.card=self.deck.pop(random.randint(0,(len(self.deck)-1)))
        self.card=self.deck.pop(0)
        print('deck remain',len(self.deck),'\n')
        return self.card
    
    def report(self):
        num=len(self.deck)
        print(f'There are {num} cards in the deck\n')
        ask=input('Show deck?\n')
        if ask.lower() in ['yes','y']:
            print('Deck :')
            print(self.deck)
            print('\n')
        else:
            print('\n')


In [2]:
deck=Deck()

In [3]:
deck.draw()

deck remain 51 



('clubs', 12)

PlayerHand class is to handle which cards are in the player hand.

In [4]:
# Player hand class
class PlayerHand():
        
    def __init__(self,owner='defult'):
        self.owner=owner
        self.hand=[]
        self.clear()
        
    def hit(self,card):
        self.hand += [card]
        print(self.owner, 'hand:',f'\n{self.hand}\n')
        
    def clear(self):
        self.hand=[]
        
    def report(self):
        num=len(self.hand)
        print(f"There are {num} cards in the {self.owner}'s hand\n")
        ask=input(f'Show {self.owner} hand?\n')
        if ask.lower() in ['yes','y']:
            print('\nHand :')
            print(self.hand)
            print('\n')
        else:
             print('\n')

In [5]:
playerhand=PlayerHand('player')
dealerhand=PlayerHand('dealer')

In [6]:
playerhand.hit(('dummy',2))
dealerhand.hit(('dummy',3))

player hand: 
[('dummy', 2)]

dealer hand: 
[('dummy', 3)]



GameController class is to handle the game mechanics.
Keep teacking the player's money
Game rounds
Dealer's rounds

In [7]:
from IPython.display import clear_output

class GameController():
    
    player_money=0
    
    # Game setting, how many money dose the player have
    def __init__(self):
        print('Welcome to Black Jack game\n')
        print('Place your bet.\n')
        print('You win, get double!\n')
        self.game_setting()        
    
    def game_setting(self):
        print('How many money are you gaming?\n')
        while True:
            try:
                self.player_money=int(input('Please enter your money: '))
                clear_output()
            except:
                print('You can not bet that, and we do not accept change.\n')
            else:
                if self.player_money<10:
                    print('Minimal is 10.')
                else:
                    print(f'Your bet {self.player_money} acceptted.\n')
                    break
        self.game()
    
    def hand_count(self,hand):
        add_up=0
        for i,v in hand:
            if v>=11:
                add_up+=10
            elif v>=2:
                add_up+=v      
        for i,v in hand:
            if v==1:
                if add_up+11 <= 21:
                    add_up+=11
                
                else:
                    add_up+=1
        return add_up
    
    def game(self):   
        
        isplaying=True
        game_state=False
        
        while isplaying:
            
            while True:
                answer = input('Are you ready? (Yes or No)\n')
                clear_output()

                if answer.lower() in ['yes','y']:
                    print('Let the game begine.\n')
                    game_state=True
                    break                    
                elif answer.lower() in ['no','n']:                    
                    print('OK, take your time.\n')
                else:
                    print('That is not an answer.\n')

            # Game start

            while game_state:
                
                # Turn reset
                turn='start'
                player_bet=0
                player_add=0
                dealer_add=0                
                deck=Deck()            
                player_hand=PlayerHand('player')            
                dealer_hand=PlayerHand('dealer')
                
                # Placing bet and checking money
                if self.player_money<=0:
                    print('\nNo more money left.\n')
                    
                    # Stop the turn to do any more
                    game_state=False
                    isplaying=False
                    
                    while True:
                        answer=input('Do you want to charge more money? (Yes or No)\n')
                        if answer.lower() in ['yes','y']:
                            self.game_setting()                            
                            break
                        elif answer.lower() in ['no','n']:
                            print('Bye.')
                            break
                        else:
                            print('That is not an answer.\n')
                else:                    
 
                    while True:
                        try:
                            print(f'You have {self.player_money}')
                            player_bet=int(input('Place your bet! : '))
                            clear_output()
                        except:
                            print('Seriously... that is not a bet.\n')
                        else:
                            if player_bet>self.player_money:
                                print("Sorry, you don't have that money.\n")
                            elif player_bet<1:
                                print('Minimal bet is 1.\n')
                            else:
                                self.player_money-=player_bet
                                print(f"Your bet is : {player_bet},",f'Money left: {self.player_money}.\n')
                                turn='player'
                                break
                
                # Player's turn
                while turn=='player':

                    if len(player_hand.hand)==0:
                        player_hand.hit(deck.draw())
                    else:
                        while True:
                            ask = input('Hit or stand? \n')
                            clear_output()
                            
                            if ask.lower() in ['hit','h']:
                                player_hand.hit(deck.draw())
                                break
                            elif ask.lower() in ['stand','s']:
                                turn='dealer'
                                break
                            else:
                                print('That is not an answer!\n')                        

                    player_add = self.hand_count(player_hand.hand)                    
                    print(f"Player's hand add up is {player_add}\n")
                    
                    if player_add>21:
                        # LOSE
                        print("Player Busted!\n")
                        turn='end'
                        
                # Dealer's turn
                while turn=='dealer':
                    
                    while dealer_add <= player_add and dealer_add!=21:
                        
                        dealer_hand.hit(deck.draw())                        
                        dealer_add=self.hand_count(dealer_hand.hand)
                        print(f"Dealer's hand add up is {dealer_add}\n")
                    
                        input('Press "Enter" to continue...')
                        clear_output()
                    # Dealer WIN
                    
                    print(f"Dealer's hand : \n{dealer_hand.hand}\n")
                    print(f"Dealer : {dealer_add}.\nPlayer : {player_add}.\n")
                    # reach here only when dealer > player
                    if dealer_add==player_add==21:
                        print('Draw!')
                        self.player_money+=player_bet
                        print(f"Here's your money {player_bet}.\nYou have {self.player_money}.\n")
                            
                    elif dealer_add <= 21:
                        print('Dealer wins!\n')
                        
                    elif dealer_add>21:    
                        print("Dealer Busted!\n")
                        self.player_money+=player_bet*2
                        print(f"Here's your money {player_bet*2}.\nYou have {self.player_money}.\n")
                        #break
                    turn='end'
                # Next round
                while turn=='end':
                    replay_answer=input('Play agin?\n')
                    if replay_answer.lower() in ['yes','y']:
                        #game_state=False
                        turn='start'
                    elif replay_answer.lower() in ['no','n']:
                        game_state=False
                        isplaying=False
                        print(f'You left with {self.player_money}.\n')
                        turn='start'
                    else:
                        print('That is not an answer.\n')           
    

In [8]:
game=GameController()

Dealer's hand : 
[('clubs', 9), ('hearts', 11), ('hearts', 8)]

Dealer : 27.
Player : 19.

Dealer Busted!

Here's your money 1200.
You have 1800.

Play agin?
n
You left with 1800.



In [9]:
game.player_money

1800